In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

# Import TensorFlow
import tensorflow as tf
from tensorflow.keras.regularizers import l1, l2

from ann_utils import *

In [20]:
preproc_folder = "enriched_shuffle"
datafolder = "../data/preproc_traintest/"+preproc_folder+'/'
output_path = "../data/models/"

prefix = 'shuffle_imp_bg_'
postfix = '_190721_1655'
trainfile = '_traindata'
testfile = '_testdata'

[X_train, y_train, feature_labels] = pd.read_pickle(datafolder+prefix+trainfile+postfix+'.pkl') 
[X_test, y_test, feature_labels] = pd.read_pickle(datafolder+prefix+testfile+postfix+'.pkl') 

In [21]:
len(feature_labels)

35

In [22]:
mlp = create_mlp_model(35, 2, [20, 5], [tf.nn.relu, tf.nn.relu],
                      optimizer = tf.keras.optimizers.Adam())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 20)                720       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 105       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 831
Trainable params: 831
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
early_stopping = TerminateOnBaseline(monitor='accuracy', baseline=0.985)
batch_size = 256
epochs=100
mlp.fit(X_train, y_train, epochs=epochs,  batch_size = batch_size, verbose=1, 
        steps_per_epoch=math.ceil(X_train.shape[0]/batch_size), callbacks=[early_stopping])
        

Epoch 1/100
181/181 [==============================] - 4s 20ms/step - loss: 0.1226 - accuracy: 0.9786 - precision: 0.3285 - recall: 0.0191
Epoch 2/100
181/181 [==============================] - 3s 18ms/step - loss: 0.0774 - accuracy: 0.9799 - precision: 0.7084 - recall: 0.0743
Epoch 3/100
181/181 [==============================] - 3s 18ms/step - loss: 0.0718 - accuracy: 0.9808 - precision: 0.7452 - recall: 0.1284
Epoch 4/100
181/181 [==============================] - 4s 22ms/step - loss: 0.0683 - accuracy: 0.9815 - precision: 0.7498 - recall: 0.1771
Epoch 5/100
181/181 [==============================] - 4s 24ms/step - loss: 0.0656 - accuracy: 0.9818 - precision: 0.7378 - recall: 0.2061
Epoch 6/100
181/181 [==============================] - 4s 24ms/step - loss: 0.0634 - accuracy: 0.9820 - precision: 0.7438 - recall: 0.2205
Epoch 7/100
181/181 [==============================] - 4s 23ms/step - loss: 0.0613 - accuracy: 0.9821 - precision: 0.7411 - recall: 0.2280
Epoch 8/100
181/181 [======

In [24]:
predictions = mlp.predict(X_test)

In [25]:
predictions.round()

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [26]:
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
fpr, tpr, thresholds = roc_curve(y_test, predictions) #false positive rate, true positive rate and thresholds
auc = roc_auc_score(y_test, predictions)
        
print('AUC: {}'.format(auc))

AUC: 0.9198259336613408
